# 05_2_обновление сети данными от 2024 года

In [1]:
import numpy as np
import pandas as pd
import networkx as nx
from matplotlib.pyplot import figure

## 1. Читаем сеществующую таблицу

In [27]:
country_attr_table = pd.read_csv('country_attr_table_2023.csv')
pd.set_option('display.max_rows', 193)
country_attr_table.set_index('Country', inplace = True)
country_attr_table

,Country index,latitude,longitude,"Population mid-year estimates (millions), 2022","Primary energy production (petajoules), 2019","GDP in current prices (millions of US dollars), 2020",Votes profile
Country,,,,,,,
AFGHANISTAN,AF,33.939110,67.709953,41.13,82.0,19793,11n111
ALBANIA,AL,41.153332,20.168331,2.84,71.0,14910,111111
ALGERIA,DZ,28.033886,1.659626,44.90,5954.0,147689,00-1000
ANDORRA,AD,42.546245,1.601554,0.08,1.0,2864,111111
ANGOLA,AO,-11.202692,17.873887,35.59,3639.0,62307,000100
ANTIGUA AND BARBUDA,AG,17.060816,-61.796428,0.09,0.0,1370,111101
ARGENTINA,AR,-38.416097,-63.616672,45.51,3292.0,383067,111111
ARMENIA,AM,40.069099,45.038189,2.78,38.0,12641,00n000
AUSTRALIA,AU,-25.274398,133.775136,26.18,18623.0,1423473,111111


## 2. Сбор данных для узлов

### 2.1 Population and migration
#### 2.1.1 Population, surface area and density

In [6]:
syb66__T02_table = pd.read_csv('reports/syb66__T02.csv', encoding='ISO-8859-1')
syb66__T02_table

,T02,"Population, density and surface area",Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,Region/Country/Area,NaN,Year,Series,Value,Footnotes,Source
1,1,"Total, all countries or areas",2010,Population mid-year estimates (millions),"6,985.60",NaN,"United Nations Population Division, New York, ..."
2,1,"Total, all countries or areas",2010,Population mid-year estimates for males (milli...,"3,514.41",NaN,"United Nations Population Division, New York, ..."
3,1,"Total, all countries or areas",2010,Population mid-year estimates for females (mil...,"3,471.20",NaN,"United Nations Population Division, New York, ..."
4,1,"Total, all countries or areas",2010,Sex ratio (males per 100 females),101.2,NaN,"United Nations Population Division, New York, ..."
...,...,...,...,...,...,...,...
7648,722,SIDS,2022,Population mid-year estimates for females (mil...,36.92,Projected estimate (medium fertility variant).,"United Nations Population Division, New York, ..."
7649,722,SIDS,2022,Sex ratio (males per 100 females),101.4,Projected estimate (medium fertility variant).,"United Nations Population Division, New York, ..."
7650,722,SIDS,2022,Population aged 0 to 14 years old (percentage),25.0,Projected estimate (medium fertility variant).,"United Nations Population Division, New York, ..."
7651,722,SIDS,2022,Population aged 60+ years old (percentage),13.3,Projected estimate (medium fertility variant).,"United Nations Population Division, New York, ..."


In [12]:
population_table = syb66__T02_table[
    (syb66__T02_table['Unnamed: 2'] == '2023') 
    &
    (
        (syb66__T02_table['Unnamed: 3'] == 'Population mid-year estimates (millions)')
    )  
][31:].drop(
    ['T02', 'Unnamed: 2', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 3'], axis = 1
).rename(
    columns={
        'Population, density and surface area': 'Country', 
        'Unnamed: 4': 'Population mid-year estimates (millions), 2022'
    }
)

#population_table.set_index(
#    'Country', inplace = True
#)

population_table

,Country,"Population mid-year estimates (millions), 2022"


In [35]:
# универсальная функция для фильтрации таблиц
def changing_names_of_index(table_index, right_index, table):
     table.rename(
        index={table_index: right_index}, 
        inplace = True
     )

#### Нет новых данных для популяции. Данные за 2022 год, как и в старой таблице -- оставляем старые данные

### 2.2 Energy

#### 2.2.1 Production, trade and supply of energy

In [13]:
syb66__T23_table = pd.read_csv('reports/syb66__T23.csv', encoding='ISO-8859-1')
syb66__T23_table

,T23,"Production, trade and supply of energy",Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,Region/Country/Area,NaN,Year,Series,Value,Footnotes,Source
1,1,"Total, all countries or areas",1995,Primary energy production (petajoules),"381,763",NaN,"United Nations Statistics Division, New York, ..."
2,1,"Total, all countries or areas",2000,Primary energy production (petajoules),"412,132",NaN,"United Nations Statistics Division, New York, ..."
3,1,"Total, all countries or areas",2005,Primary energy production (petajoules),"477,281",NaN,"United Nations Statistics Division, New York, ..."
4,1,"Total, all countries or areas",2010,Primary energy production (petajoules),"530,045",NaN,"United Nations Statistics Division, New York, ..."
...,...,...,...,...,...,...,...
8580,716,Zimbabwe,2010,Supply per capita (gigajoules),31,NaN,"United Nations Statistics Division, New York, ..."
8581,716,Zimbabwe,2015,Supply per capita (gigajoules),32,NaN,"United Nations Statistics Division, New York, ..."
8582,716,Zimbabwe,2018,Supply per capita (gigajoules),31,NaN,"United Nations Statistics Division, New York, ..."
8583,716,Zimbabwe,2019,Supply per capita (gigajoules),30,NaN,"United Nations Statistics Division, New York, ..."


In [14]:
energy_table = syb66__T23_table[
    (syb66__T23_table['Unnamed: 2'] == '2020') 
    #& 
    #(syb65__T23_table['Production, trade and supply of energy'] == 'Zimbabwe')
    &
    (
        (syb66__T23_table['Unnamed: 3'] == 'Primary energy production (petajoules)')
    )    
][7:].drop(
    ['T23', 'Unnamed: 2', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 3'], axis = 1
).rename(
    columns={
        'Production, trade and supply of energy': 'Country', 
        'Unnamed: 4': 'Primary energy production (petajoules), 2020'
    }
)

energy_table

,Country,"Primary energy production (petajoules), 2020"
288,Afghanistan,71
328,Albania,63
367,Algeria,"5,506"
403,American Samoa,0
435,Andorra,1
...,...,...
8406,Viet Nam,"2,509"
8446,Wallis and Futuna Islands,0
8478,Yemen,164
8513,Zambia,405


In [23]:
# Поднимаем регистр в названии стран
energy_table['Country'] = [x.upper() for x in energy_table['Country']]

In [24]:
energy_table.set_index(
    'Country', inplace = True
)
energy_table

,"Primary energy production (petajoules), 2020"
Country,
AFGHANISTAN,71
ALBANIA,63
ALGERIA,"5,506"
AMERICAN SAMOA,0
ANDORRA,1
...,...
VIET NAM,"2,509"
WALLIS AND FUTUNA ISLANDS,0
YEMEN,164


In [25]:
# проверка на целостность данных
energy_table[energy_table['Primary energy production (petajoules), 2020'].isna()]

,"Primary energy production (petajoules), 2020"
Country,


In [38]:
#country_attr_table[10:200]

In [41]:
# приведение название стран к общим
changing_attr_arr = [
    ['BOLIVIA (PLURIN. STATE OF)', 'BOLIVIA (PLURINATIONAL STATE OF)'],
    ['CÔTE DIVOIRE',"CÔTE D'IVOIRE"],
    ["DEM. PEOPLE'S REP. KOREA", "DEMOCRATIC PEOPLE'S REPUBLIC OF KOREA"],
    ["DEM. REP. OF THE CONGO", 'DEMOCRATIC REPUBLIC OF THE CONGO'],
    ["LAO PEOPLE'S DEM. REP.", "LAO PEOPLE'S DEMOCRATIC REPUBLIC"],
    ['MICRONESIA (FED. STATES OF)', 'MICRONESIA (FEDERATED STATES OF)'],
    ['SAINT VINCENT & GRENADINES', 'SAINT VINCENT AND THE GRENADINES'],
    ['TURKIYE', 'TÜRKIYE'],
    ['UNITED REP. OF TANZANIA', 'UNITED REPUBLIC OF TANZANIA'],
    ['UNITED STATES OF AMERICA', 'UNITED STATES'],
    ['VENEZUELA (BOLIV. REP. OF)', 'VENEZUELA (BOLIVARIAN REPUBLIC OF)'],
    ['NETHERLANDS (KINGDOM OF THE)', 'NETHERLANDS']
]

for item in changing_attr_arr:
    changing_names_of_index(item[0], item[1], energy_table)
    
#population_table.loc[[changing_attr_arr[-1][1]], list(population_table.columns)]

In [42]:
# присоединяем колонку к основной таблице
country_attr_table_temp = pd.merge(
    country_attr_table, 
    energy_table, 
    how='outer', 
    left_index=True, 
    right_index=True
)#.dropna(
    #subset=['latitude']
#)#.to_csv(
    #'country_attr_table.csv'
#) 
country_attr_table_temp

,Country index,latitude,longitude,"Population mid-year estimates (millions), 2022","Primary energy production (petajoules), 2019","GDP in current prices (millions of US dollars), 2020",Votes profile,"Primary energy production (petajoules), 2020"
Country,,,,,,,,
AFGHANISTAN,AF,33.939110,67.709953,41.13,82.0,19793.0,11n111,71
ALBANIA,AL,41.153332,20.168331,2.84,71.0,14910.0,111111,63
ALGERIA,DZ,28.033886,1.659626,44.90,5954.0,147689.0,00-1000,"5,506"
AMERICAN SAMOA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
ANDORRA,AD,42.546245,1.601554,0.08,1.0,2864.0,111111,1
...,...,...,...,...,...,...,...,...
VIET NAM,VN,14.058324,108.277199,98.19,2776.0,271158.0,00-1000,"2,509"
WALLIS AND FUTUNA ISLANDS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
YEMEN,YE,15.552727,48.516388,33.70,113.0,27958.0,110101,164


In [69]:
# проверка на соответствие стран
country_attr_table_temp[
    country_attr_table_temp['Country index'].notna()
    &
    country_attr_table_temp['Primary energy production (petajoules), 2020'].isna()
]

,Country index,latitude,longitude,"Population mid-year estimates (millions), 2022","Primary energy production (petajoules), 2019","GDP in current prices (millions of US dollars), 2020",Votes profile,"Primary energy production (petajoules), 2020"
Country,,,,,,,,
MONACO,MC,43.750298,7.412841,0.04,NaN,6816.0,111111,NaN
SAN MARINO,SM,43.942360,12.457777,0.03,NaN,1555.0,111111,NaN


In [61]:
#country_attr_table.loc[['MONACO', 'SAN MARINO']]

In [72]:
#energy_table[100:200]

In [68]:
# проверка на целостность
len(country_attr_table_temp[
    country_attr_table_temp['Country index'].notna()
])

193

In [78]:
country_attr_table = country_attr_table_temp[
    country_attr_table_temp['Country index'].notna()
].copy()

pd.set_option('display.max_rows', 6)
country_attr_table

,Country index,latitude,longitude,"Population mid-year estimates (millions), 2022","Primary energy production (petajoules), 2019","GDP in current prices (millions of US dollars), 2020",Votes profile,"Primary energy production (petajoules), 2020"
Country,,,,,,,,
AFGHANISTAN,AF,33.939110,67.709953,41.13,82.0,19793.0,11n111,71
ALBANIA,AL,41.153332,20.168331,2.84,71.0,14910.0,111111,63
ALGERIA,DZ,28.033886,1.659626,44.90,5954.0,147689.0,00-1000,"5,506"
...,...,...,...,...,...,...,...,...
YEMEN,YE,15.552727,48.516388,33.70,113.0,27958.0,110101,164
ZAMBIA,ZM,-13.133897,27.849332,20.02,391.0,18111.0,11n111,405
ZIMBABWE,ZW,-19.015438,29.154857,16.32,415.0,21787.0,00-10-10,377


### 2.3 Валовой внутренний продукт и валовой внутренний продукт на душу населения
#### Gross domestic product and gross domestic product per capita

In [102]:
syb66__T13_table = pd.read_csv('reports/syb66__T13.csv', encoding='ISO-8859-1')
syb66__T13_table

,T13,Gross domestic product and gross domestic product per capita,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,Region/Country/Area,NaN,Year,Series,Value,Footnotes,Source
1,1,"Total, all countries or areas",1995,GDP in current prices (millions of US dollars),"31,269,041",NaN,"United Nations Statistics Division, New York, ..."
2,1,"Total, all countries or areas",2005,GDP in current prices (millions of US dollars),"47,775,404",NaN,"United Nations Statistics Division, New York, ..."
...,...,...,...,...,...,...,...
6773,716,Zimbabwe,2019,GDP real rates of growth (percent),-6.1,NaN,"United Nations Statistics Division, New York, ..."
6774,716,Zimbabwe,2020,GDP real rates of growth (percent),-5.3,NaN,"United Nations Statistics Division, New York, ..."
6775,716,Zimbabwe,2021,GDP real rates of growth (percent),6.3,NaN,"United Nations Statistics Division, New York, ..."


In [117]:
GDP_table = syb66__T13_table[
    (syb66__T13_table['Unnamed: 2'] == '2021') 
    #& 
    #(syb65__T23_table['Production, trade and supply of energy'] == 'Zimbabwe')
    &
    (
        (syb66__T13_table['Unnamed: 3'] == 'GDP in current prices (millions of US dollars)')
    )    
][30:].drop(
    ['T13', 'Unnamed: 2', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 3'], axis = 1
).rename(
    columns={
        'Gross domestic product and gross domestic product per capita': 'Country', 
        'Unnamed: 4': 'GDP in current prices (millions of US dollars), 2021'
    }
)

GDP_table

,Country,"GDP in current prices (millions of US dollars), 2021"
847,Afghanistan,"14,939"
875,Albania,"18,260"
903,Algeria,"163,473"
...,...,...
6698,Zambia,"21,313"
6726,Zanzibar,"2,080"
6754,Zimbabwe,"24,118"


In [118]:
GDP_table['Country'] = [x.upper() for x in GDP_table['Country']]

In [119]:
GDP_table.set_index(
    'Country', inplace = True
)
GDP_table

,"GDP in current prices (millions of US dollars), 2021"
Country,
AFGHANISTAN,"14,939"
ALBANIA,"18,260"
ALGERIA,"163,473"
...,...
ZAMBIA,"21,313"
ZANZIBAR,"2,080"
ZIMBABWE,"24,118"


In [120]:
# приведение название стран к общим
changing_attr_arr = [
    ['BOLIVIA (PLURIN. STATE OF)', 'BOLIVIA (PLURINATIONAL STATE OF)'],
    ['CÔTE DIVOIRE',"CÔTE D'IVOIRE"],
    ["DEM. PEOPLE'S REP. KOREA", "DEMOCRATIC PEOPLE'S REPUBLIC OF KOREA"],
    ["DEM. REP. OF THE CONGO", 'DEMOCRATIC REPUBLIC OF THE CONGO'],
    ["LAO PEOPLE'S DEM. REP.", "LAO PEOPLE'S DEMOCRATIC REPUBLIC"],
    ['MICRONESIA (FED. STATES OF)', 'MICRONESIA (FEDERATED STATES OF)'],
    ['SAINT VINCENT & GRENADINES', 'SAINT VINCENT AND THE GRENADINES'],
    ['TURKIYE', 'TÜRKIYE'],
    ['UNITED REP. OF TANZANIA', 'UNITED REPUBLIC OF TANZANIA'],
    ['UNITED STATES OF AMERICA', 'UNITED STATES'],
    ['VENEZUELA (BOLIV. REP. OF)', 'VENEZUELA (BOLIVARIAN REPUBLIC OF)'],
    ['NETHERLANDS (KINGDOM OF THE)', 'NETHERLANDS']
]

for item in changing_attr_arr:
    changing_names_of_index(item[0], item[1], GDP_table)

In [121]:
GDP_table

,"GDP in current prices (millions of US dollars), 2021"
Country,
AFGHANISTAN,"14,939"
ALBANIA,"18,260"
ALGERIA,"163,473"
...,...
ZAMBIA,"21,313"
ZANZIBAR,"2,080"
ZIMBABWE,"24,118"


In [113]:
# присоединяем колонку к основной таблице
country_attr_table_temp = pd.merge(
    country_attr_table, 
    GDP_table, 
    how='outer', 
    left_index=True, 
    right_index=True
)#.dropna(
    #subset=['latitude']
#)#.to_csv(
    #'country_attr_table.csv'
#) 
country_attr_table_temp

,Country index,latitude,longitude,"Population mid-year estimates (millions), 2022","Primary energy production (petajoules), 2019","GDP in current prices (millions of US dollars), 2020",Votes profile,"Primary energy production (petajoules), 2020","GDP in current prices (millions of US dollars), 2021"
Country,,,,,,,,,
AFGHANISTAN,AF,33.939110,67.709953,41.13,82.0,19793.0,11n111,71,"14,939"
ALBANIA,AL,41.153332,20.168331,2.84,71.0,14910.0,111111,63,"18,260"
ALGERIA,DZ,28.033886,1.659626,44.90,5954.0,147689.0,00-1000,"5,506","163,473"
...,...,...,...,...,...,...,...,...,...
ZAMBIA,ZM,-13.133897,27.849332,20.02,391.0,18111.0,11n111,405,"21,313"
ZANZIBAR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,080"
ZIMBABWE,ZW,-19.015438,29.154857,16.32,415.0,21787.0,00-10-10,377,"24,118"


In [123]:
# проверка на соответствие стран
country_attr_table_temp[
    country_attr_table_temp['Country index'].notna()
    &
    country_attr_table_temp['GDP in current prices (millions of US dollars), 2021'].isna()
]

,Country index,latitude,longitude,"Population mid-year estimates (millions), 2022","Primary energy production (petajoules), 2019","GDP in current prices (millions of US dollars), 2020",Votes profile,"Primary energy production (petajoules), 2020","GDP in current prices (millions of US dollars), 2021"
Country,,,,,,,,,


In [124]:
# проверка на целостность
len(country_attr_table_temp[
    country_attr_table_temp['Country index'].notna()
])

193

In [125]:
country_attr_table = country_attr_table_temp[
    country_attr_table_temp['Country index'].notna()
].copy()

pd.set_option('display.max_rows', 6)
country_attr_table

,Country index,latitude,longitude,"Population mid-year estimates (millions), 2022","Primary energy production (petajoules), 2019","GDP in current prices (millions of US dollars), 2020",Votes profile,"Primary energy production (petajoules), 2020","GDP in current prices (millions of US dollars), 2021"
Country,,,,,,,,,
AFGHANISTAN,AF,33.939110,67.709953,41.13,82.0,19793.0,11n111,71,"14,939"
ALBANIA,AL,41.153332,20.168331,2.84,71.0,14910.0,111111,63,"18,260"
ALGERIA,DZ,28.033886,1.659626,44.90,5954.0,147689.0,00-1000,"5,506","163,473"
...,...,...,...,...,...,...,...,...,...
YEMEN,YE,15.552727,48.516388,33.70,113.0,27958.0,110101,164,"9,947"
ZAMBIA,ZM,-13.133897,27.849332,20.02,391.0,18111.0,11n111,405,"21,313"
ZIMBABWE,ZW,-19.015438,29.154857,16.32,415.0,21787.0,00-10-10,377,"24,118"


In [128]:
type(country_attr_table['GDP in current prices (millions of US dollars), 2021'][1])

str

In [615]:
#country_attr_table_3[column_name][112]
#not(isinstance(country_attr_table_3[column_name][112], float))

In [590]:
#country_attr_table_3[isinstance(country_attr_table_3[column_name], float)]

In [130]:
# привеодим знначение GPT к float
column_name = 'GDP in current prices (millions of US dollars), 2021'

#country_attr_table_3[column_name] = str(country_attr_table_3[column_name]) 

#change_type_val(column_name)
country_attr_table[
        column_name
    ] = [
        #x.replace(',', '') for x in country_attr_table_3[column_name] if not(isinstance(x, float))
        x if isinstance(x, float) else x.replace(',', '') for x in country_attr_table[column_name] 
    ]
    
# Преобразовать столбец к типу данных numpy.float64
country_attr_table[column_name] = country_attr_table[column_name].astype(np.float64)
#country_attr_table[column_name] = pd.to_numeric(country_attr_table[column_name]) 

In [136]:
type(country_attr_table[column_name][2])

numpy.float64

In [137]:
country_attr_table

,Country index,latitude,longitude,"Population mid-year estimates (millions), 2022","Primary energy production (petajoules), 2019","GDP in current prices (millions of US dollars), 2020",Votes profile,"Primary energy production (petajoules), 2020","GDP in current prices (millions of US dollars), 2021"
Country,,,,,,,,,
AFGHANISTAN,AF,33.939110,67.709953,41.13,82.0,19793.0,11n111,71,14939.0
ALBANIA,AL,41.153332,20.168331,2.84,71.0,14910.0,111111,63,18260.0
ALGERIA,DZ,28.033886,1.659626,44.90,5954.0,147689.0,00-1000,"5,506",163473.0
...,...,...,...,...,...,...,...,...,...
YEMEN,YE,15.552727,48.516388,33.70,113.0,27958.0,110101,164,9947.0
ZAMBIA,ZM,-13.133897,27.849332,20.02,391.0,18111.0,11n111,405,21313.0
ZIMBABWE,ZW,-19.015438,29.154857,16.32,415.0,21787.0,00-10-10,377,24118.0


In [3]:
country_attr_table.to_csv('country_attr_table_2024.csv')

In [4]:
#pd.set_option('display.max_rows', 200)
country_attr_table = pd.read_csv('country_attr_table_2024.csv')
country_attr_table

,Country,Country index,latitude,longitude,"Population mid-year estimates (millions), 2022","Primary energy production (petajoules), 2019","GDP in current prices (millions of US dollars), 2020",Votes profile,"Primary energy production (petajoules), 2020","GDP in current prices (millions of US dollars), 2021"
0,AFGHANISTAN,AF,33.939110,67.709953,41.13,82.0,19793.0,11n111,71,14939.0
1,ALBANIA,AL,41.153332,20.168331,2.84,71.0,14910.0,111111,63,18260.0
2,ALGERIA,DZ,28.033886,1.659626,44.90,5954.0,147689.0,00-1000,"5,506",163473.0
3,ANDORRA,AD,42.546245,1.601554,0.08,1.0,2864.0,111111,1,3325.0
4,ANGOLA,AO,-11.202692,17.873887,35.59,3639.0,62307.0,000100,"3,253",70533.0
...,...,...,...,...,...,...,...,...,...,...
188,VENEZUELA (BOLIVARIAN REPUBLIC OF),VE,6.423750,-66.589730,28.30,3238.0,106359.0,nnnnnn,"2,070",111813.0
189,VIET NAM,VN,14.058324,108.277199,98.19,2776.0,271158.0,00-1000,"2,509",366138.0
190,YEMEN,YE,15.552727,48.516388,33.70,113.0,27958.0,110101,164,9947.0
191,ZAMBIA,ZM,-13.133897,27.849332,20.02,391.0,18111.0,11n111,405,21313.0


# Добавление параметров узлов в сеть

In [27]:
#df[1][1]
#G = nx.read_gexf('UN_UKRAINE--POLITICAL_CONDITIONS_.gexf')

#list(G.nodes(data=True))[0]

('AFGHANISTAN', {'label': 'AFGHANISTAN'})

In [24]:
nodes_array_un = []

for index in range (0, len(country_attr_table)):
    nodes_array_un.append(
        (
            country_attr_table['Country'][index], 
         {
             #'latitude': country_attr_table['latitude'][index],
             #'longitude': country_attr_table['longitude'][index],
             #'Population': country_attr_table['Population mid-year estimates (millions), 2022'][index],
             'Energy': country_attr_table['Primary energy production (petajoules), 2020'][index],
             'GDP': country_attr_table['GDP in current prices (millions of US dollars), 2021'][index],
             #'CI': country_attr_table['Country index'][index],
         }
        ) 
    )

In [28]:
G_3 = nx.Graph()
G_3.add_nodes_from(nodes_array_un)

In [37]:
#G2 = nx.read_gexf('UN_UKRAINE--POLITICAL_CONDITIONS_attr_2023.gexf')
G2 = nx.read_gexf('UN_UKRAINE--POLITICAL_CONDITIONS_attr_25_03_23.gexf')

In [38]:
len(G2)

193

In [39]:
list(G2.nodes(data=True))[0]

('AFGHANISTAN',
 {'votes_profile': '11n111',
  'latitude': 33.93911,
  'longitude': 67.709953,
  'Population': 41.13,
  'Energy': 82.0,
  'GDP': 19793,
  'CI': 'AF',
  'sol_group': 'none',
  'label': 'AFGHANISTAN'})

In [40]:
R = nx.compose(G2, G_3)

In [42]:
list(R.nodes(data=True))[2]

('ALGERIA',
 {'votes_profile': '00-1000',
  'latitude': 28.033886,
  'longitude': 1.6596259999999998,
  'Population': 44.9,
  'Energy': '5,506',
  'GDP': 163473.0,
  'CI': 'DZ',
  'sol_group': '00-1000',
  'label': 'ALGERIA'})

In [43]:
nx.write_gexf(R, 'UN_UKRAINE--POLITICAL_CONDITIONS_attr_11_04_2024.gexf')

In [44]:
len(R)

193